<a href="https://colab.research.google.com/github/simodepth/Keyword-Research/blob/main/Google_Autosuggest_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🚀 Run Google Autosuggest to find keywords ideas in bulk 


---

The following framework uses `urllib` and `requests_html` Python libraries to scrape Google Autosuggest and retrieve endless chunks of keywords combinations produced by term suffixes and content angles of your choice.




#Assumptions

- Please note the framework is not set on a specific target location, so the output does not provide local results
- The output is messed with unwanted **punctuations**. You will be able to copy and paste the output onto a txt file, thus into a spreadsheet where you want to play around with filters to phase the punctuation.

**Should you find a way to clean up the output, please feel free to fork up the last part of the script!** 

In [ ]:
!pip install requests_html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.7 MB/s 
     |████████████████████████████████| 138 kB 62.5 MB/s 
     |████████████████████████████████| 112 kB 66.7 MB/s 
     |████████████████████████████████| 127 kB 72.7 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=0c2c88408aafea90cb1e9daad36e653ca0c7506d80f5ad4e5ead7a75aa331465
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=060607d11c159bc8a535deedd11090c47345091e583247588d3090b75a31c71b
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
#@title Run Import Modules
import requests
import urllib
import json
import operator
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession


#Set up connections with Google SERPs

In [ ]:
def get_source(url):

    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    except requests.exceptions.RequestException as e:
        print(e)

In [ ]:
def get_results(query):
    query = urllib.parse.quote_plus(query)
    response = get_source("https://suggestqueries.google.com/complete/search?output=chrome&hl=en&q=" + query)
    results = json.loads(response.text)
    return results

In [ ]:
#@title Type your Keyword
search_term = "retirement villages" #@param {type:"string"}
results = get_results(search_term)
results

['retirement villages',
 ['retirement villages texas',
  'retirement villages north carolina',
  'retirement villages near me',
  'retirement villages in florida',
  'retirement villages in arizona',
  'retirement villages in south carolina',
  'retirement villages by the sea',
  'retirement villages uk'],
 ['', '', '', '', '', '', '', ''],
 [],
 {'google:clientdata': {'bpc': False, 'tlw': False},
  'google:suggestrelevance': [601, 600, 555, 554, 553, 552, 551, 550],
  'google:suggestsubtypes': [[402],
   [402],
   [512],
   [512],
   [512],
   [512],
   [512],
   [512]],
  'google:suggesttype': ['QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY'],
  'google:verbatimrelevance': 1300}]

#Let's format the results

In [ ]:
def format_results(results):
    suggestions = []
    for index, value in enumerate(results[1]):
        suggestion = {'term': value, 'relevance': results[4]['google:suggestrelevance'][index]}
        suggestions.append(suggestion)
    return suggestions

In [ ]:
formatted_results = format_results(results)
formatted_results


[{'relevance': 601, 'term': 'retirement villages texas'},
 {'relevance': 600, 'term': 'retirement villages north carolina'},
 {'relevance': 555, 'term': 'retirement villages near me'},
 {'relevance': 554, 'term': 'retirement villages in florida'},
 {'relevance': 553, 'term': 'retirement villages in arizona'},
 {'relevance': 552, 'term': 'retirement villages in south carolina'},
 {'relevance': 551, 'term': 'retirement villages by the sea'},
 {'relevance': 550, 'term': 'retirement villages uk'}]

🔗 **"Relevance"**  refers to an automated estimation based on the frequency that specific term is subjected to in the search results page

#Spice-up the research by adding some term suffixes 🌶

In [ ]:
def get_expanded_term_suffixes():
    expanded_term_suffixes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
                             'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    return expanded_term_suffixes

#Define your Content Angle according to the target funnel stage

In [ ]:
def get_expanded_term_prefixes():
    expanded_term_prefixes = ['what *', 'where *', 'how to *', 'why *', 'buy*', 'how much*',
                              'best *', 'worse *', 'rent*', 'sale*', 'offer*','vs*','or*'
                             ]
    return expanded_term_prefixes 

Content angle default's definition: 


```
# ['what *', 'where *', 'how to *', 'why *', 'vs*', 'or*', 'buy*', 'how much*',
                              'best *', 'worse *', 'tutorial *', 'tips *', 'ideas *', 'review *', 'guide *' 
                             ]
```



#Make sure to expand the search 

In [ ]:
def get_expanded_terms(query):

    expanded_term_prefixes = get_expanded_term_prefixes()
    expanded_term_suffixes = get_expanded_term_suffixes()   

    terms = []
    terms.append(query)

    for term in expanded_term_prefixes:
        terms.append(term + ' ' + query)

    for term in expanded_term_suffixes:
        terms.append(query + ' ' + term)

    return terms


In [ ]:
get_expanded_terms(search_term)


['retirement villages',
 'what * retirement villages',
 'where * retirement villages',
 'how to * retirement villages',
 'why * retirement villages',
 'buy* retirement villages',
 'how much* retirement villages',
 'best * retirement villages',
 'worse * retirement villages',
 'rent* retirement villages',
 'sale* retirement villages',
 'offer* retirement villages',
 'vs* retirement villages',
 'or* retirement villages',
 'retirement villages a',
 'retirement villages b',
 'retirement villages c',
 'retirement villages d',
 'retirement villages e',
 'retirement villages f',
 'retirement villages g',
 'retirement villages h',
 'retirement villages i',
 'retirement villages j',
 'retirement villages k',
 'retirement villages l',
 'retirement villages m',
 'retirement villages n',
 'retirement villages o',
 'retirement villages p',
 'retirement villages q',
 'retirement villages r',
 'retirement villages s',
 'retirement villages t',
 'retirement villages u',
 'retirement villages v',
 'ret

#Look for further suggestions




In [ ]:
def get_expanded_suggestions(query):

    all_results = []

    expanded_terms = get_expanded_terms(query)
    for term in expanded_terms:
        results = get_results(term)
        results = format_results(results)
        all_results = all_results + results
        all_results = sorted(all_results, key=lambda k: k['relevance'], reverse=True)
    return all_results


In [ ]:
expanded_results = get_expanded_suggestions(search_term)
expanded_results

[{'relevance': 1251, 'term': 'retirement villages oregon'},
 {'relevance': 1251, 'term': 'retirement villages queensland'},
 {'relevance': 1251, 'term': 'retirement villages tennessee'},
 {'relevance': 1251, 'term': 'retirement villages victoria'},
 {'relevance': 1251, 'term': 'retirement villages yorkshire'},
 {'relevance': 1250, 'term': 'retirement villages arizona'},
 {'relevance': 1250, 'term': 'retirement villages by the sea'},
 {'relevance': 1250, 'term': 'retirement villages colorado'},
 {'relevance': 1250, 'term': 'retirement villages delaware'},
 {'relevance': 1250, 'term': 'retirement villages florida'},
 {'relevance': 1250, 'term': 'retirement villages group'},
 {'relevance': 1250, 'term': 'retirement villages in florida'},
 {'relevance': 1250,
  'term': 'retirement villages johannesburg northern suburbs'},
 {'relevance': 1250, 'term': 'retirement villages las vegas'},
 {'relevance': 1250, 'term': 'retirement villages montana'},
 {'relevance': 1250, 'term': 'retirement villa